# Project: Reproducing the Paper "Improving clinical outcome predictions using convolution over medical entities with multimodal learning"

Team 34: Kristine Cheng (cycheng4), Vanessa Chen (zhenc5), Sophia Yu (sophiay3)

## Project Location
* Google Drive Link: https://drive.google.com/drive/folders/1nlDMRbCBY27ygu5EKwvnyUR3SDempmbJ?usp=drive_link

* Github Link: https://github.com/zhenc5/CS598-Group-Project


# Mount Notebook to Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Introduction

*   Background of the problem

  It is crucial to assess a patient’s health by looking at their medical tests and predicting how they might fare during their stay in the ICU. The type of problem addressed in the paper is clinical outcome prediction, specifically predicting mortality (in-hospital & in-ICU) and length of ICU stay (LOS) (>3 days & >7 days).

  ![task_prediction.png](https://drive.google.com/uc?export=view&id=1Mfvjp_7vDHGOAF5SqUyOnfMb2tQQuhUM)
  
  Predicting clinical outcomes is an important problem in healthcare, because it can help hospitals and healthcare providers reduce healthcare costs, improve patient outcomes by determining treatment methods, and optimize healthcare resource utilization. By accurately predicting LOS and mortality, healthcare providers can provide targeted interventions to those at high risk, thereby leading to better patient outcomes and more efficient use of hospital resources.

  One of the major difficulties associated with predicting these clinical outcomes using electronic health records (EHR) is in standardizing the preprocessing steps, such as in the handling of missing data and outliers, unit conversions, and the transformation of raw data into usable features to be used in deep learning algorithms [1].  Additionally, previous studies that aim to predict these clinical outcomes have used only structured patient data, such as historical patient diagnoses (ICD codes) [5, 6], lab results and other measurements taken in the ICU [7-9]. To improve the accuracy of the predictions, unstructured clinical notes can be added to the deep learning model. However, extracting medical entities from unstructured clinical notes presents its own challenges because it is free text usually containing grammatical errors, shorthand, medical jargon and redundant information [1].

  Some state-of-the-art deep learning algorithm methods for using EHR data to predict clinical outcomes include Long Short Term Memory (LSTM) and Gated Recurrent Unit (GRU) because they are most effective at learning from sequence data. Lipton et al. demonstrated the effectiveness of a LSTM to model clinical data, specifically to classify 128 diagnoses using 13 clinical measurements [10]. Choi et al. showed promising results in predicting multi-label diagnosis for a patient's next visit using a GRU-based model called DoctorAI [5].

*   Paper explanation

  Electronic Health Record (EHR) data is commonly used in deep learning applications for clinical outcome predictions. However, traditional approaches often overlook the unstructured data within EHR, such as clinical notes and radiology. Bardak and Tan address [1]  this issue by exploring methods to improve two different common risk prediction tasks - mortality and length of ICU stay (LOS). The paper proposes a deep learning method that involves extracting medical entities from clinical notes and integrating them into prediction models using a convolution-based multimodal architecture. Additionally, they evaluated different embedding techniques, such as Word2Vec and FastText on medical entities.

  The innovative feature in the proposed method in the paper is the use of CNN architecture to capture local patterns in the EHR data and medical entity embeddings of the clinical notes, and then to combine the learned features from the CNN with features extracted from the timeseries data to make its predictions.

  The results show that the proposed method outperforms the baseline models on all 4 clinical outcome predictions in terms of AUCROC, AURPRC, and F1 score, with the exception of LOS >7 days where the F1 score was greater for the baseline model.

  Overall, the paper makes an important contribution to the research regime of clinical outcome predictions by introducing a novel approach that not only enhances the accuracy of these predictions but also has the adaptability to be applied to other clinical outcome prediction tasks. The implementation of convolution on medical entities, extracted from EHR clinical notes, in conjunction with multimodal learning, signifies an important step forward in the development of predictive models for clinical outcomes.


# Scope of Reproducibility:

Below lists the hypotheses to be tested and the corresponding experiments that will be run:
  1. The proposed convolution-based multimodal architecture outperforms the baseline multimodal architecture for each of the 4 clinical outcome prediction tasks.
  * Various embedding techniques, such as Word2Vec, FastText, and the concatenation of Word2Vec and FastText embeddings on the EHR clinical notes, will also be compared among the baseline multimodel architecture and the proposed convolution-based architecture.

  2. The baseline multimodal model shows an improved prediction performance compared to the baseline time-series GRU and LSTM models on each of the 4 clinical outcome tasks.
  * Various embedding techniques, such as Doc2Vec, Word2Vec, FastText, and the concatenation of Word2Vec and FastText embeddings on the EHR clinical notes, will also be compared among the baseline models.



# Methodology

#### Import the necessary packages

In [ ]:
# import packages
from google.colab import drive
import os
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')

import re
import spacy
from nltk import sent_tokenize, word_tokenize, punkt

from gensim.models import Word2Vec, FastText
#import glove
#from glove import Corpus

import collections
import gc

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, Activation, Concatenate, LSTM, GRU
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, Convolution1D
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D, MaxPool1D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.backend import set_session, clear_session, get_session

import tensorflow as tf

from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

from logging import NullHandler

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


##  Data
In order to implement the paper's code, 3 folders must first be created in the directory and named "data", "embeddings", and "results".

#### Links to the data sources:
* Download the MIMIC-III dataset via https://mimic.physionet.org/
* MIMIC-Extract implementation: https://github.com/MLforHealth/MIMIC_Extract
* med7 implementation: https://github.com/kormilitzin/med7
* Download Pre-trained Word2Vec embeddings: https://github.com/kexinhuang12345/clinicalBERT
* Preprocessing Script: https://github.com/kaggarwal/ClinicalNotesICU

#### Source of the data:

The data is collected from running MIMIC-III data [2-4] through MIMIC-Extract Pipeline. Since we only wished to use the output of this pipeline, we were able to directly download a preprocessed version with default parameters from their Github page [4, 11]. The dataset is stored in the "data" folder as all_hourly_data.h5.


#### Statistics:

For the time series data:

The MIMIC-III dataset contains EHR data of 58,976 unique hospital admissions and 61,532 ICU admissions from 46,520 patients.

The MIMIC-Extract dataset contains a patient's first ICU visit and already eliminates patients with ages < 15 years and where the LOS is not between 12 hours and 10 days [1]. It contains 34,472 patients and 104 time-series variables.

Then, we drop any patients who do not have at least 30 hours of data. We also drop any clinical notes that do not contain chart time information and any patients that do not have any clinical notes in 24 hours. This leads to a final cohort, after clinical note elimination, of 23,944 records of patients, hospital admissions, and ICU admissions.

For the medical entities data:

There will be 7 medical entities (Drug, Strength, Form, Route, Dosage, Frequency, Duration) with the final unique counts being 18268, 10749, 597, 1193, 7239, 3344, and 1185 respectfully.

#### Data process:

By feeding data through first 24 hour features, the data should be split into three different csv files named ADMISSION, NOTEEVENTS, ICUSTAYS respectfully and placed in the "data" folder.

The medical entities from the clinical notes will be used to enhance the prediction performance. In order to extract the medical embeddings, we used a pre-trained clinical named-entity recognition (NER) model, med7 [12], which extracts 7 different entities (Drug, Strength, Duration, Route, Form, Dosage, Frequency). Then, we used the pre-trained Word2Vec and FastText embedding techniques [13] (stored in the "embeddings" folder) to convert the medical entities into word representations.

The train/valid/test split, for all clinical tasks, is based on class
distribution with 70%/10%/20% ratio.


### Check Time Series Data and Collect the Patient ID

In [ ]:
DATAPATH = '/content/drive/MyDrive/CS598_Project/data'
lvl2_train_imputer = pd.read_pickle(os.path.join(DATAPATH, "lvl2_imputer_train.pkl"))
lvl2_dev_imputer = pd.read_pickle(os.path.join(DATAPATH, "lvl2_imputer_dev.pkl"))
lvl2_test_imputer = pd.read_pickle(os.path.join(DATAPATH,"lvl2_imputer_test.pkl"))
Ys = pd.read_pickle(os.path.join(DATAPATH, "Ys.pkl"))

print("Shape of train, dev, test {}, {}, {}.".format((lvl2_train_imputer.shape), (lvl2_dev_imputer.shape), (lvl2_test_imputer.shape)))
print("After applying time series feature (24 hours), train, dev, and test statistic: {}, {}, {}".format((lvl2_train_imputer.shape[0] / 24), (lvl2_dev_imputer.shape[0] / 24), (lvl2_test_imputer.shape[0] / 24)))

patients_ids = []
for entry in Ys.index:
  patients_ids.append(entry[0])

print("Number of total patient {}".format(len(patients_ids)))

Shape of train, dev, test (402240, 312), (57456, 312), (114960, 312).
After applying time series feature (24 hours), train, dev, and test statistic: 16760.0, 2394.0, 4790.0
Number of total patient 23944


#### Extracting Time-Series Features and Preprocessing Clinical Notes

This step was run separately and the ouput "preprocessed_notes.p" was uploaded to the data folder. The code is provided in "03-Preprocessing-Clinical-Notes"


#### Extract Medical Entities in Clinical Notes
This step was executed locally on a PC, and the resulting output file "ner_df.p" was uploaded to the data folder. The code for this process is provided in the notebook titled "04 - Apply-med7-on-Clinical-Notes.ipynb."

Given that "preprocessed_notes" comprises over 200,000 rows of data, we opted to extract medical entities solely from the first 5000 entries for the draft submission. However, for the final submission, we aim to expand the extraction process to include more entries.

#### Represent Entities with Different Embeddings

This step was run locally on a pc and the output "new_ner_word2vec_dict.pkl" was uploaded to the data folder. The code is provided in "05_Represent_Entities_With_Different_Embeddings.ipynb."


We have currently integrated Word2Vec embeddings for training the timeseries baseline model. Our next steps involve incorporating FastText and combined embeddings for both the multimodal baseline model and the proposed model.

#### Create Timeseries Data

This step was run locally on a pc and the output was uploaded to the data folder. The code is provided in "06_Create_Timeseries_Data.ipynb."

##   Model
We have successfully implemented the timeseries baseline model, which runs and prints performance metrics such as AUC, AUPRC, accuracy (ACC), and F1 score. To demonstrate implementation progress, we have temporarily reduced the epoch number from 100 as stated in the original paper to 3 in this notebook.

Our next steps involve completing the implementation of the Multimodal baseline model and the proposed model.

### 1. Timeseries Baseline

We use Long Short Term Memory (LSTM) and Gated Recurrent Units (GRU) and compare their AUC and AUPRC performances in capturing temporal information between patient features.

In [ ]:
# Reset Keras Session
def reset_keras(model):
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    gc.collect() # if it's done something you should see a number being outputted

def make_prediction_timeseries(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_timeseries(predictions, probs, ground_truth, model_name,
                problem_type, iteration, hidden_unit_size, type_of_ner):

    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)


    result_dict = {}
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1


    file_name = str(hidden_unit_size)+"-"+model_name+"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+".p"

    result_path = "/content/drive/MyDrive/CS598_Project/results/"
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))

    print("AUC: {}, AUPRC: {}, Accuracy: {}, F1 Score: {}".format(auc, auprc, acc, F1))

#def l2_regularizer(scale):
#    return tf.keras.regularizers.l2(scale)

def timeseries_model(layer_name, number_of_unit):
    K.clear_session()

    sequence_input = Input(shape=(24,104),  name = "timeseries_input")

    if layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)
    else:
        x = GRU(number_of_unit)(sequence_input)

    #logits_regularizer = tf.keras.regularizers.l2(0.01)
    logits_regularizer = keras.regularizers.l2(0.01)
    sigmoid_pred = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=tf.keras.initializers.GlorotUniform(),
                  kernel_regularizer=logits_regularizer)(x)


    model = Model(inputs=sequence_input, outputs=sigmoid_pred)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

type_of_ner = "new"

x_train_lstm = pd.read_pickle(os.path.join(DATAPATH, type_of_ner+"_x_train.pkl"))
x_dev_lstm = pd.read_pickle(os.path.join(DATAPATH, type_of_ner+"_x_dev.pkl"))
x_test_lstm = pd.read_pickle(os.path.join(DATAPATH, type_of_ner+"_x_test.pkl"))

y_train = pd.read_pickle(os.path.join(DATAPATH, type_of_ner+"_y_train.pkl"))
y_dev = pd.read_pickle(os.path.join(DATAPATH, type_of_ner+"_y_dev.pkl"))
y_test = pd.read_pickle(os.path.join(DATAPATH, type_of_ner+"_y_test.pkl"))

epoch_num = 3
model_patience = 3
monitor_criteria = 'val_loss'
batch_size = 128

unit_sizes = [128, 256]
#unit_sizes = [256]
iter_num = 11
target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']
layers = ["GRU"]
#layers = ["LSTM", "GRU"]
for each_layer in layers:
    print("Layer: ", each_layer)
    for each_unit_size in unit_sizes:
        print("Hidden unit: ", each_unit_size)
        for iteration in range(1, iter_num):
            print("Iteration number: ", iteration)
            print("=============================")

            for each_problem in target_problems:
                print ("Problem type: ", each_problem)
                print ("__________________")


                early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                best_model_name = str(each_layer)+"-"+str(each_unit_size)+"-"+str(each_problem)+"-"+"best_model.keras"
                checkpoint = ModelCheckpoint(best_model_name, monitor='val_loss', verbose=1,
                    save_best_only=True, mode='min')


                callbacks = [early_stopping_monitor, checkpoint]

                model = timeseries_model(each_layer, each_unit_size)
                model.fit(x_train_lstm, y_train[each_problem], epochs=epoch_num, verbose=1,
                          validation_data=(x_dev_lstm, y_dev[each_problem]), callbacks=callbacks, batch_size= batch_size)

                model.load_weights(best_model_name)

                probs, predictions = make_prediction_timeseries(model, x_test_lstm)
                save_scores_timeseries(predictions, probs, y_test[each_problem].values,str(each_layer),
                                       each_problem, iteration, each_unit_size,type_of_ner)
                reset_keras(model)
                #del model
                clear_session()
                gc.collect()


Layer:  GRU
Hidden unit:  128
Iteration number:  1
Problem type:  mort_hosp
__________________
Epoch 1/3
1/2 [==============>...............] - ETA: 3s - loss: 0.6795 - acc: 0.6484
Epoch 1: val_loss improved from inf to 0.62753, saving model to GRU-128-mort_hosp-best_model.keras
2/2 [==============================] - 4s 749ms/step - loss: 0.6665 - acc: 0.6467 - val_loss: 0.6275 - val_acc: 0.7179
Epoch 2/3
1/2 [==============>...............] - ETA: 0s - loss: 0.5797 - acc: 0.7422
Epoch 2: val_loss improved from 0.62753 to 0.61716, saving model to GRU-128-mort_hosp-best_model.keras
2/2 [==============================] - 0s 118ms/step - loss: 0.5678 - acc: 0.7500 - val_loss: 0.6172 - val_acc: 0.6410
Epoch 3/3
1/2 [==============>...............] - ETA: 0s - loss: 0.5246 - acc: 0.7734
Epoch 3: val_loss improved from 0.61716 to 0.61372, saving model to GRU-128-mort_hosp-best_model.keras
2/2 [==============================] - 0s 11ms/step
AUC: 0.7659574468085106, AUPRC: 0.39722222222222214,


Epoch 1: val_loss improved from inf to 0.73184, saving model to GRU-128-los_7-best_model.keras
2/2 [==============================] - 3s 689ms/step - loss: 0.6763 - acc: 0.6304 - val_loss: 0.7318 - val_acc: 0.4615
Epoch 2/3
1/2 [==============>...............] - ETA: 0s - loss: 0.5813 - acc: 0.8047
Epoch 2: val_loss improved from 0.73184 to 0.69803, saving model to GRU-128-los_7-best_model.keras
2/2 [==============================] - 0s 124ms/step - loss: 0.5649 - acc: 0.8152 - val_loss: 0.6980 - val_acc: 0.5897
Epoch 3/3
1/2 [==============>...............] - ETA: 0s - loss: 0.4945 - acc: 0.8828
Epoch 3: val_loss improved from 0.69803 to 0.67656, saving model to GRU-128-los_7-best_model.keras
2/2 [==============================] - 0s 12ms/step
AUC: 0.46808510638297873, AUPRC: 0.12302621867077436, Accuracy: 0.7307692307692307, F1 Score: 0.2222222222222222
Iteration number:  2
Problem type:  mort_hosp
__________________
Epoch 1/3
1/2 [==============>...............] - ETA: 2s - loss: 0


Epoch 1: val_loss improved from inf to 0.68054, saving model to GRU-128-mort_hosp-best_model.keras
2/2 [==============================] - 3s 659ms/step - loss: 0.7392 - acc: 0.5543 - val_loss: 0.6805 - val_acc: 0.5641
Epoch 2/3
2/2 [==============================] - ETA: 0s - loss: 0.6202 - acc: 0.7500
Epoch 2: val_loss improved from 0.68054 to 0.64475, saving model to GRU-128-mort_hosp-best_model.keras
2/2 [==============================] - 0s 151ms/step - loss: 0.6202 - acc: 0.7500 - val_loss: 0.6447 - val_acc: 0.5641
Epoch 3/3
2/2 [==============================] - ETA: 0s - loss: 0.5406 - acc: 0.7772
Epoch 3: val_loss improved from 0.64475 to 0.62373, saving model to GRU-128-mort_hosp-best_model.keras
2/2 [==============================] - 0s 11ms/step
AUC: 0.7404255319148937, AUPRC: 0.3565891472868217, Accuracy: 0.75, F1 Score: 0.3157894736842105
Problem type:  mort_icu
__________________
Epoch 1/3
1/2 [==============>...............] - ETA: 3s - loss: 0.7495 - acc: 0.4766
Epoch 

### 2. Multimodal Baseline

We use the pre-trained NER model, med7, to extract different named medical entities and represent them with two different embedding methods - word embedding and document embedding.

In [ ]:
class MultimodalModelTrainer:
    def __init__(self, type_of_ner):
        self.type_of_ner = type_of_ner

    def reset_keras(self, model):
        clear_session()
        gc.collect()

    def create_dataset(self, dict_of_ner):
        temp_data = []
        for k, v in sorted(dict_of_ner.items()):
            temp = []
            for embed in v:
                temp.append(embed)
            temp_data.append(np.mean(temp, axis=0))
        return np.asarray(temp_data)

    def make_prediction_multi_avg(self, model, test_data):
        probs = model.predict(test_data)
        y_pred = [1 if i>=0.5 else 0 for i in probs]
        return probs, y_pred

    def save_scores_multi_avg(self, predictions, probs, ground_truth,
                              embed_name, problem_type, iteration, hidden_unit_size,
                              sequence_name):
        auc = roc_auc_score(ground_truth, probs)
        auprc = average_precision_score(ground_truth, probs)
        acc   = accuracy_score(ground_truth, predictions)
        F1    = f1_score(ground_truth, predictions)

        result_dict = {}
        result_dict['auc'] = auc
        result_dict['auprc'] = auprc
        result_dict['acc'] = acc
        result_dict['F1'] = F1

        result_path = "results/"
        file_name = f"{sequence_name}-{hidden_unit_size}-{embed_name}-{problem_type}-{iteration}-{self.type_of_ner}-avg-.p"
        pd.to_pickle(result_dict, os.path.join(result_path, file_name))

        print(auc, auprc, acc, F1)

    def avg_ner_model(self, layer_name, number_of_unit, embedding_name):
        if embedding_name == "concat":
            input_dimension = 200
        else:
            input_dimension = 100

        sequence_input = Input(shape=(24,104))
        input_avg = Input(shape=(input_dimension, ), name="avg")

        if layer_name == "GRU":
            x = GRU(number_of_unit)(sequence_input)
        elif layer_name == "LSTM":
            x = LSTM(number_of_unit)(sequence_input)

        x = Concatenate()([x, input_avg])
        x = Dense(256, activation='relu')(x)
        x = Dropout(0.2)(x)

        logits_regularizer = tf.keras.regularizers.l2(0.01)
        preds = Dense(1, activation='sigmoid', use_bias=False,
                      kernel_initializer=tf.keras.initializers.GlorotUniform(),
                      kernel_regularizer=logits_regularizer)(x)

        opt = Adam(lr=0.001, decay=0.01)
        model = Model(inputs=[sequence_input, input_avg], outputs=preds)
        model.compile(loss='binary_crossentropy',
                      optimizer=opt,
                      metrics=['acc'])

        return model

    def train_models(self):
        x_train_lstm = pd.read_pickle(f"data/{self.type_of_ner}_x_train.pkl")
        x_dev_lstm = pd.read_pickle(f"data/{self.type_of_ner}_x_dev.pkl")
        x_test_lstm = pd.read_pickle(f"data/{self.type_of_ner}_x_test.pkl")

        y_train = pd.read_pickle(f"data/{self.type_of_ner}_y_train.pkl")
        y_dev = pd.read_pickle(f"data/{self.type_of_ner}_y_dev.pkl")
        y_test = pd.read_pickle(f"data/{self.type_of_ner}_y_test.pkl")

        ner_word2vec = pd.read_pickle(f"data/{self.type_of_ner}_ner_word2vec_limited_dict.pkl")
        ner_fasttext = pd.read_pickle(f"data/{self.type_of_ner}_ner_fasttext_limited_dict.pkl")
        ner_concat = pd.read_pickle(f"data/{self.type_of_ner}_ner_combined_limited_dict.pkl")

        train_ids = pd.read_pickle(f"data/{self.type_of_ner}_train_ids.pkl")
        dev_ids = pd.read_pickle(f"data/{self.type_of_ner}_dev_ids.pkl")
        test_ids = pd.read_pickle(f"data/{self.type_of_ner}_test_ids.pkl")

        embedding_types = ['word2vec', 'fasttext', 'concat']
        embedding_dict = [ner_word2vec, ner_fasttext, ner_concat]
        target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']

        num_epoch = 100
        model_patience = 5
        monitor_criteria = 'val_loss'
        batch_size = 64
        iter_num = 2
        unit_sizes = [128, 256]

        layers = ["GRU"]
        for each_layer in layers:
            print ("Layer: ", each_layer)
            for each_unit_size in unit_sizes:
                print ("Hidden unit: ", each_unit_size)

                for embed_dict, embed_name in zip(embedding_dict, embedding_types):
                    print ("Embedding: ", embed_name)
                    print("=============================")

                    temp_train_ner = dict((k, ner_word2vec[k]) for k in train_ids)
                    temp_dev_ner = dict((k, ner_word2vec[k]) for k in dev_ids)
                    temp_test_ner = dict((k, ner_word2vec[k]) for k in test_ids)

                    x_train_ner = self.create_dataset(temp_train_ner)
                    x_dev_ner = self.create_dataset(temp_dev_ner)
                    x_test_ner = self.create_dataset(temp_test_ner)

                    for iteration in range(1, iter_num):
                        print ("Iteration number: ", iteration)

                        for each_problem in target_problems:
                            print ("Problem type: ", each_problem)
                            print ("__________________")

                            early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                            best_model_name = f"avg-{embed_name}-{each_problem}-best_model.hdf5"
                            checkpoint = ModelCheckpoint(best_model_name, monitor='val_loss', verbose=1,
                                                         save_best_only=True, mode='min', period=1)

                            callbacks = [early_stopping_monitor, checkpoint]

                            model = self.avg_ner_model(each_layer, each_unit_size, embed_name)

                            model.fit([x_train_lstm, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1,
                                      validation_data=([x_dev_lstm, x_dev_ner], y_dev[each_problem]), callbacks=callbacks,
                                      batch_size=batch_size )

                            model.load_weights(best_model_name)

                            probs, predictions = self.make_prediction_multi_avg(model, [x_test_lstm, x_test_ner])

                            self.save_scores_multi_avg(predictions, probs, y_test[each_problem], embed_name, each_problem,
                                                        iteration, each_unit_size, each_layer)

                            self.reset_keras(model)

### 3. Proposed Model

We leverage 1D Convolutional Neural Networks to extract features from medical entities, subsequently integrating them with recurrent and fully-connected layers for comprehensive patient representation

In [ ]:
class ProposedModel:
    def __init__(self, type_of_ner):
        self.type_of_ner = type_of_ner

    def make_prediction_cnn(self, model, test_data):
        probs = model.predict(test_data)
        y_pred = [1 if i>=0.5 else 0 for i in probs]
        return probs, y_pred

    def save_scores_cnn(self, predictions, probs, ground_truth,
                        embed_name, problem_type, iteration, hidden_unit_size,
                        sequence_name):
        auc = roc_auc_score(ground_truth, probs)
        auprc = average_precision_score(ground_truth, probs)
        acc   = accuracy_score(ground_truth, predictions)
        F1    = f1_score(ground_truth, predictions)

        result_dict = {}
        result_dict['auc'] = auc
        result_dict['auprc'] = auprc
        result_dict['acc'] = acc
        result_dict['F1'] = F1

        result_path = "results/cnn/"
        file_name = f"{sequence_name}-{hidden_unit_size}-{embed_name}-{problem_type}-{iteration}-{self.type_of_ner}-cnn-.p"
        pd.to_pickle(result_dict, os.path.join(result_path, file_name))

        print(auc, auprc, acc, F1)

    def proposedmodel(self, layer_name, number_of_unit, embedding_name, ner_limit, num_filter):
        if embedding_name == "concat":
            input_dimension = 200
        else:
            input_dimension = 100

        sequence_input = Input(shape=(24,104))
        input_img = Input(shape=(ner_limit, input_dimension), name="cnn_input")

        text_conv1d = Conv1D(filters=num_filter, kernel_size=3,
                             padding='valid', strides=1, dilation_rate=1, activation='relu',
                             kernel_initializer=tf.contrib.layers.xavier_initializer())(input_img)

        text_conv1d = Conv1D(filters=num_filter*2, kernel_size=3,
                             padding='valid', strides=1, dilation_rate=1, activation='relu',
                             kernel_initializer=tf.contrib.layers.xavier_initializer())(text_conv1d)

        text_conv1d = Conv1D(filters=num_filter*3, kernel_size=3,
                             padding='valid', strides=1, dilation_rate=1, activation='relu',
                             kernel_initializer=tf.contrib.layers.xavier_initializer())(text_conv1d)

        text_embeddings = GlobalMaxPooling1D()(text_conv1d)

        if layer_name == "GRU":
            x = GRU(number_of_unit)(sequence_input)
        elif layer_name == "LSTM":
            x = LSTM(number_of_unit)(sequence_input)

        concatenated = Concatenate()([x, text_embeddings])
        concatenated = Dense(512, activation='relu')(concatenated)
        concatenated = Dropout(0.2)(concatenated)

        logits_regularizer = tf.contrib.layers.l2_regularizer(scale=0.01)
        preds = Dense(1, activation='sigmoid', use_bias=False,
                             kernel_initializer=tf.contrib.layers.xavier_initializer(),
                             kernel_regularizer=logits_regularizer)(concatenated)

        opt = Adam(lr=1e-3, decay=0.01)
        model = Model(inputs=[sequence_input, input_img], outputs=preds)
        model.compile(loss='binary_crossentropy',
                      optimizer=opt,
                      metrics=['acc'])

        return model

    def train_models(self):
        # Loading data and setting up parameters
        # (x_train_lstm, x_dev_lstm, x_test_lstm, y_train, y_dev, y_test, ner_word2vec, ner_fasttext,
        #  ner_concat, train_ids, dev_ids, test_ids) should be defined before calling this method

        embedding_types = ['word2vec', 'fasttext', 'concat']
        embedding_dict = [ner_word2vec, ner_fasttext, ner_concat]
        target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']

        num_epoch = 100
        model_patience = 5
        monitor_criteria = 'val_loss'
        batch_size = 64
        filter_number = 32
        ner_representation_limit = 64

        sequence_model = "GRU"
        sequence_hidden_unit = 256

        maxiter = 11

        for embed_dict, embed_name in zip(embedding_dict, embedding_types):
            print ("Embedding: ", embed_name)
            print("=============================")

            temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
            tem_dev_ner = dict((k, embed_dict[k]) for k in dev_ids)
            temp_test_ner = dict((k, embed_dict[k]) for k in test_ids)

            x_train_dict = {}
            x_dev_dict = {}
            x_test_dict = {}

            x_train_dict = self.get_subvector_data(ner_representation_limit, embed_name, temp_train_ner)
            x_dev_dict = self.get_subvector_data(ner_representation_limit, embed_name, tem_dev_ner)
            x_test_dict = self.get_subvector_data(ner_representation_limit, embed_name, temp_test_ner)

            x_train_dict_sorted = collections.OrderedDict(sorted(x_train_dict.items()))
            x_dev_dict_sorted = collections.OrderedDict(sorted(x_dev_dict.items()))
            x_test_dict_sorted = collections.OrderedDict(sorted(x_test_dict.items()))

            x_train_ner = np.asarray(x_train_dict_sorted.values())
            x_dev_ner = np.asarray(x_dev_dict_sorted.values())
            x_test_ner = np.asarray(x_test_dict_sorted.values())

            for iteration in range(1,maxiter):
                print ("Iteration number: ", iteration)

                for each_problem in target_problems:
                    print ("Problem type: ", each_problem)
                    print ("__________________")

                    early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                    best_model_name = f"{ner_representation_limit}-basiccnn1d-{embed_name}-{each_problem}-best_model.hdf5"
                    checkpoint = ModelCheckpoint(best_model_name, monitor=monitor_criteria, verbose=1,
                                                 save_best_only=True, mode='min')

                    reduce_lr = ReduceLROnPlateau(monitor=monitor_criteria, factor=0.2,
                                      patience=2, min_lr=0.00001, epsilon=1e-4, mode='min')

                    callbacks = [early_stopping_monitor, checkpoint, reduce_lr]

                    model = self.proposedmodel(sequence_model, sequence_hidden_unit,
                                               embed_name, ner_representation_limit, filter_number)
                    model.fit([x_train_lstm, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1,
                              validation_data=([x_dev_lstm, x_dev_ner], y_dev[each_problem]), callbacks=callbacks, batch_size=batch_size)

                    probs, predictions = self.make_prediction_cnn(model, [x_test_lstm, x_test_ner])
                    self.print_scores_cnn(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration, sequence_hidden_unit)

                    model.load_weights(best_model_name)

                    probs, predictions = self.make_prediction_cnn(model, [x_test_lstm, x_test_ner])
                    self.save_scores_cnn(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration,
                                        sequence_hidden_unit, sequence_model)
                    del model
                    clear_session()
                    gc.collect()

# Training

The paper suggests the computational requirements are:  
  NVIDIA Tesla K80 GPU with 24 GB of VRAM, 378 GB of RAM and Intel Xeon E5 2683 processor.

Because the MIMIC-III dataset is so large, we ran the data preprocessing part locally on a CPU computer with 16 GB of VRAM, 32GB of RAM and Intel i9-12900H processor.

The training results are saved in the results folder.

# Evaluation

We will be using 3 different statistical methods for the comparison of our models.
* Area Under the Receiver Operating Characteristic curve (AUROC), which is the area under the true positive rate versus the false positive rate.
* Area Under the Precision-Recall Curve (AUPRC), which is the area under
the precision versus recall plot.
* F1 score measures accuracy by considering both precision and recall to compute the score, providing a balance between false positives and false negatives.

In [ ]:
# show evaluation results

#Results from Timeseries Baseline Model
# List all files in the results directory
result_path = "/content/drive/MyDrive/CS598_Project/results/"
result_files = os.listdir(result_path)

results_dfs = []

# Loop through each result file
for file_name in result_files:

    # Load the result dictionary from the file
    result_dict = pd.read_pickle(os.path.join(result_path, file_name))

    # Extract relevant information from the file name
    parts = file_name.split("-")
    model = parts[0]
    problem_type = parts[2]
    iteration = int(parts[3])

    # Extract performance metrics from the result dictionary
    auc = result_dict["auc"]
    auprc = result_dict["auprc"]
    accuracy = result_dict["acc"]
    f1 = result_dict["F1"]

    # Append the results to the DataFrame
    result_df = pd.DataFrame({
        "Model": [model],
        "Problem Type": [problem_type],
        "Iteration": [iteration],
        "AUC": [auc],
        "AUPRC": [auprc],
        "Accuracy": [accuracy],
        "F1": [f1]
    })

    results_dfs.append(result_df)

results_df = pd.concat(results_dfs, ignore_index=True)

# Display the DataFrame with the results
print(results_df)

    Model Problem Type  Iteration       AUC     AUPRC  Accuracy        F1
0     128    mort_hosp          1  0.812766  0.297210  0.884615  0.400000
1     128     mort_icu          1  0.768707  0.239683  0.903846  0.285714
2     128        los_3          1  0.571875  0.442925  0.634615  0.424242
3     128        los_7          1  0.497872  0.111536  0.826923  0.000000
4     128    mort_hosp          2  0.748936  0.410932  0.846154  0.333333
..    ...          ...        ...       ...       ...       ...       ...
155   256        los_7          9  0.608511  0.132026  0.634615  0.095238
156   256    mort_hosp         10  0.859574  0.566667  0.884615  0.571429
157   256     mort_icu         10  0.687075  0.167057  0.846154  0.333333
158   256        los_3         10  0.682813  0.581843  0.673077  0.514286
159   256        los_7         10  0.412766  0.102592  0.750000  0.133333

[160 rows x 7 columns]


# Results
We anticipate our results to be similar to the ones given in the paper. The baseline multimodal model should perform better overall than the baseline GRU model across all 4 clinical task predictions. The proposed model should perform better overall across all 4 clinical task predictions compared to the baseline multimodal model.

This section will be updated after we have implemented all our models.

In [ ]:
# metrics to evaluate my model

# plot figures to better show the results

# it is better to save the numbers and figures for your presentation.

## Model comparison

We will discuss how our results compared to the results given in the paper. We will complete this section after training the model and gathering the model prediction results for all 4 clinical tasks.

In [ ]:
# compare you model with others
# you don't need to re-run all other experiments, instead, you can directly refer the metrics/numbers in the paper

# Discussion

The paper is so far reproducible. We will later assess reproducibility of paper when we have finished training the models and compared the model prediction results.

What was easy during the reproduction:
* all the code was well-documented, which made it easy to find and use

What was difficult during the reproduction:
* the duration of preprocessing the data was long
* retrieving the clinical entities took longer than expected so we had to use a smaller subset of the preprocessing data
* downloading large datasets (i.e. MIMIC-III) and gathering the pre-trained models from various sources took a longer time than we thought it would
* Fasttext embedding was not available in original repository download link given by the author. The embedding was found in the comments section of the Issues tab in the paper's Github repository. This was very difficult to find.
* in the original code, several packages were outdated or deprecated, leading to issues with compatibility and functionality.


Suggestions on how to improve the reproducibility:
* a note on the duration of each step would have been helpful, i.e. preprocessing data, employing the embedding techniques, training the model, etc.

What we would do in the next phase:
* can add more feature to improve prediction performance, such as drug ICD codes.
* can compare results of using context-dependent word embeddings (i.e. BERT) with context-independent word embeddings (i.e. Word2Vec and FastText) for representing medical entities.


## Plans
* Due to time constraints, we kept the template code for mounting a google drive. We will change this to using a public link to access the data in the future.
* extract medical entities using word embedding techniques on the entire dataset, or a larger subset of the dataset.
* represent extracted medical entities
* train the models. If training takes a long time, we will train on a small # of epoch (like 3). Alternatively, we will train the model in a different jupyter notebook and just load the pretrained model to this notebook.
* Display the figures and metrics resulting from model prediction
* Write a function to display model prediction summary of results and figures.



# References

1. Bardak B, Tan M, "Improving clinical outcome predictions using convolution over medical entities with multimodal learning", Artificial Intelligence in Medicine, 2021, 117:0933-3657, doi:https://doi.org/10.1016/j.artmed.2021.102112.
2. Johnson A, Pollard T, Mark R, "MIMIC-III Clinical Database (version 1.4)", PhysioNet, 2016, doi:https://doi.org/10.13026/C2XW26.
3. Johnson AEW, Pollard TJ, Shen L, Lehman LH, Feng M, Ghassemi M, Moody B, Szolovits P, Celi L A, Mark RG, "MIMIC-III, a freely accessible critical care database", Scientific Data, 2016, 3:160035.
4. Goldberger A, Amaral L, Glass L, Hausdorff J, Ivanov PC, Mark R, ... & Stanley HE, "PhysioBank, PhysioToolkit, and PhysioNet: Components of a new research resource for complex physiologic signals", Circulation [Online], 2000, 101:23, pp. e215–e220.
5. Choi E, Bahadori MT, Schuetz A, Stewart WF, Sun J. Doctor AI: predicting clinical events via recurrent neural networks. Machine learning for healthcare conference 2016:301-18.
6. Choi E, Bahadori MT, Sun J, Kulas J, Schuetz A, Stewart W. Retain: an interpretable predictive model for healthcare using reverse time attention mechanism. Advances in neural information processing systems. 2016. p. 3504-12.
7. Caballero Barajas KL, Akella R. Dynamically modeling patient’s health state from electronic medical records: a time series approach. Proceedings of the 21st ACM SIGKDD international conference on knowledge discovery and data mining 2015:69–78.
8. Song H, Rajan D, Thiagarajan JJ, Spanias A. Attend and diagnose: clinical time series analysis using attention models. Thirty-second AAAI conference on artificial intelligence 2018.
9. Suresh H, Gong JJ, Guttag JV. Learning tasks for multitask learning: heterogenous patient populations in the ICU. Proceedings of the 24th ACM SIGKDD international conference on knowledge discovery & data mining 2018:802–10.
10. Lipton ZC, Kale DC, Elkan C, Wetzel R. Learning to diagnose with LSTM recurrent neural networks. 2015 (arXiv preprint), arXiv:1511.03677.
11. Wang S, McDermott MBA, Chauhan G, Hughes MC, Naumann T, Ghassemi M. MIMIC-Extract: A Data Extraction, Preprocessing, and Representation
Pipeline for MIMIC-III. arXiv:1907.08322.
12. Kormilitzin A, Vaci N, Liu Q, Nevado-Holgado A. Med7: A Transferable Clinical Natural Language Processing Model for Electronic Health Records. 2020. arXiv:2003.01271.
13. Huang K, Altosaar J, Ranganath R. ClinicalBERT: Modeling Clinical Notes and Predicting Hospital Readmission. 2019. arXiv:1904.05342

